In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [4]:
def obter_detalhes_livro(url_livro):
    r = requests.get(url_livro)
    soup = BeautifulSoup(r.text, "html.parser")

    tabela = soup.find("table", class_="table table-striped")
    dados = {}
    for row in tabela.find_all("tr"):
        chave = row.th.text.strip()
        valor = row.td.text.strip()
        dados[chave] = valor

    descricao_tag = soup.find("meta", attrs={"name": "description"})
    descricao = descricao_tag["content"].strip() if descricao_tag else ""

    categoria = soup.find("ul", class_="breadcrumb").find_all("li")[2].text.strip()

    return {
        "UPC": dados.get("UPC", ""),
        "Preço sem imposto": dados.get("Price (excl. tax)", ""),
        "Preço com imposto": dados.get("Price (incl. tax)", ""),
        "Disponibilidade (qtd)": re.findall(r"\d+", dados.get("Availability", "0"))[0],
        "Descrição": descricao,
        "Categoria": categoria
    }

def coletar_livros_completos():
    livros = []
    base_url = "https://books.toscrape.com/catalogue/page-{}.html"
    site_url = "https://books.toscrape.com/catalogue/"

    for page in range(1, 25):
        print(f"Coletando página {page}")
        url = base_url.format(page)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")

        for item in soup.select(".product_pod"):
            titulo = item.h3.a["title"]
            preco_texto = item.select_one(".price_color").text
            preco = float(re.sub(r"[^\d.]", "", preco_texto))

            disponibilidade = item.select_one(".availability").text.strip()
            rating_str = item.p["class"][1]
            ratings_dict = {
                "One": 1,
                "Two": 2,
                "Three": 3,
                "Four": 4,
                "Five": 5
            }
            avaliacao = ratings_dict.get(rating_str, 0)

            href = item.h3.a["href"]
            link_livro = site_url + href.replace("../", "")
            detalhes = obter_detalhes_livro(link_livro)

            livros.append({
                "Título": titulo,
                "Preço": preco,
                "Avaliação (1-5)": avaliacao,
                "Disponibilidade": disponibilidade,
                "Categoria": detalhes["Categoria"],
                "Descrição": detalhes["Descrição"],
                "Preço sem imposto": detalhes["Preço sem imposto"],
                "Preço com imposto": detalhes["Preço com imposto"],
                "Quantidade disponível": detalhes["Disponibilidade (qtd)"],
                "UPC": detalhes["UPC"],
                "Link": link_livro
            })

            time.sleep(0.2)  

    return pd.DataFrame(livros)

In [7]:
df = coletar_livros_completos()
df.to_csv("livros_completos.csv", index=False)
print(f"Total de livros coletados: {df.shape[0]}")
df.head()

Coletando página 1
Coletando página 2
Coletando página 3
Coletando página 4
Coletando página 5
Coletando página 6
Coletando página 7
Coletando página 8
Coletando página 9
Coletando página 10
Coletando página 11
Coletando página 12
Coletando página 13
Coletando página 14
Coletando página 15
Coletando página 16
Coletando página 17
Coletando página 18
Coletando página 19
Coletando página 20
Coletando página 21
Coletando página 22
Coletando página 23
Coletando página 24
Total de livros coletados: 480


,Título,Preço,Avaliação (1-5),Disponibilidade,Categoria,Descrição,Preço sem imposto,Preço com imposto,Quantidade disponível,UPC,Link
0,A Light in the Attic,51.77,3,In stock,Poetry,It's hard to imagine a world without A Light i...,Â£51.77,Â£51.77,22,a897fe39b1053632,https://books.toscrape.com/catalogue/a-light-i...
1,Tipping the Velvet,53.74,1,In stock,Historical Fiction,"""Erotic and absorbing...Written with starling ...",Â£53.74,Â£53.74,20,90fa61229261140a,https://books.toscrape.com/catalogue/tipping-t...
2,Soumission,50.10,1,In stock,Fiction,"Dans une France assez proche de la nÃ´tre, un ...",Â£50.10,Â£50.10,20,6957f44c3847a760,https://books.toscrape.com/catalogue/soumissio...
3,Sharp Objects,47.82,4,In stock,Mystery,"WICKED above her hipbone, GIRL across her hear...",Â£47.82,Â£47.82,20,e00eb4fd7b871a48,https://books.toscrape.com/catalogue/sharp-obj...
4,Sapiens: A Brief History of Humankind,54.23,5,In stock,History,From a renowned historian comes a groundbreaki...,Â£54.23,Â£54.23,20,4165285e1663650f,https://books.toscrape.com/catalogue/sapiens-a...
